In [1]:
import os
import sys
import json

# Use the current working directory instead of __file__
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..','parser'))

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from antlr4 import *
from cobol85.Cobol85Lexer import Cobol85Lexer
from cobol85.Cobol85Parser import Cobol85Parser
from cobol85.Cobol85Listener import Cobol85Listener

print("Import Successful!")

Import Successful!


In [2]:
class CobolCFGListener(Cobol85Listener):
    def __init__(self):
        self.nodes = []
        self.edges = []
        self.business_rules = {}

    def enterProcedureDivision(self, ctx: Cobol85Parser.ProcedureDivisionContext):
        # Start processing procedure division for control flow
        self.nodes.append({"id": "start", "data": {"label": "start"}})


    def enterAcceptStatement(self, ctx: Cobol85Parser.AcceptStatementContext):
        # Capture an ACCEPT statement
        accept_node = {"id": "1", "data": {"label": f"ACCEPT {ctx.identifier().getText()}\n"}}
        self.nodes.append(accept_node)
        self.edges.append({"id": "1", "source": "start", "target": "1", "label": "start"})

    def enterIfStatement(self, ctx: Cobol85Parser.IfStatementContext):
        # Capture an IF statement in control flow
        if_node = {"id": "3", "data": {"label": f"IF {ctx.condition().getText()}\n"}}
        self.nodes.append(if_node)
        self.edges.append({"id": "3", "source": "2", "target": "3", "label": "sequential next"})

In [3]:
input_code = """
IDENTIFICATION DIVISION.
       PROGRAM-ID. EXAMPLE.
       DATA DIVISION.
       WORKING-STORAGE SECTION.
       77  NUM         PICTURE 99.
       77  QUOTIENT    PICTURE 99.
       77  REMAIN      PICTURE 9.
       PROCEDURE DIVISION.
           ACCEPT NUM.
           DIVIDE NUM BY 2 GIVING QUOTIENT REMAINDER REMAIN.
           IF REMAIN = 0
                   DISPLAY NUM ' IS EVEN'
           ELSE
                   DISPLAY  NUM ' IS ODD'
           END-IF.
           STOP RUN.
"""

lexer = Cobol85Lexer(InputStream(input_code))
stream = CommonTokenStream(lexer)
parser = Cobol85Parser(stream)
tree = parser.startRule()

In [4]:
cfg_listener = CobolCFGListener()
walker = ParseTreeWalker()
walker.walk(cfg_listener, tree)

print("Nodes:", cfg_listener.nodes)
print("Edges:", cfg_listener.edges)
print("Business Rules:", cfg_listener.business_rules)

Nodes: [{'id': 'start', 'data': {'label': 'start'}}, {'id': '1', 'data': {'label': 'ACCEPT NUM\n'}}, {'id': '3', 'data': {'label': 'IF REMAIN=0\n'}}]
Edges: [{'id': '1', 'source': 'start', 'target': '1', 'label': 'start'}, {'id': '3', 'source': '2', 'target': '3', 'label': 'sequential next'}]
Business Rules: {}


In [5]:
data = {
    "nodes": cfg_listener.nodes,
    "edges": cfg_listener.edges,
}

data

{'nodes': [{'id': 'start', 'data': {'label': 'start'}},
  {'id': '1', 'data': {'label': 'ACCEPT NUM\n'}},
  {'id': '3', 'data': {'label': 'IF REMAIN=0\n'}}],
 'edges': [{'id': '1', 'source': 'start', 'target': '1', 'label': 'start'},
  {'id': '3', 'source': '2', 'target': '3', 'label': 'sequential next'}]}

In [6]:
with open('cfg.json', 'w') as f:
    json.dump(data, f)